# 📬 Email Automation for Food Insecurity (Jupyter Notebook Version)

This notebook sends personalized emails and checks for 'yes' responses from patients.

- Sends emails using Gmail SMTP
- Checks inbox for replies
- Saves updated response to a new `.xlsx` file
- Supports optional auto-scheduler (runs `check_responses()` every 10 minutes)

In [ ]:
# ✅ Setup - Imports & Config
import pandas as pd
import smtplib
import imaplib
import email
from email.message import EmailMessage
from datetime import datetime
import os
import time
from threading import Thread

# ✉️ Email credentials
EMAIL_ADDRESS = ""
EMAIL_PASSWORD = ""
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
IMAP_SERVER = "imap.gmail.com"
EXCEL_FILE = "sample.xlsx"

In [3]:
def load_patients():
    try:
        df = pd.read_excel(EXCEL_FILE)
        df.dropna(how="all", inplace=True)
        print("[INFO] Excel file loaded successfully.")
        return df
    except Exception as e:
        print(f"[ERROR] Failed to load Excel file: {e}")
        return pd.DataFrame()


def save_patients(df, prefix="updated"):
    try:
        base, ext = os.path.splitext(EXCEL_FILE)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        new_file = f"{prefix}_{base}{ext}"
        df.to_excel(new_file, index=False)
        print(f"[INFO] Data saved to {os.path.abspath(new_file)}")
    except Exception as e:
        print(f"[ERROR] Failed to save Excel file: {e}")


In [4]:
def send_emails():
    df = load_patients()
    if df.empty:
        print("[ERROR] No data found in Excel.")
        return

    sent_count = 0
    fail_count = 0

    try:
        print("[INFO] Connecting to SMTP server...")
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT, timeout=10) as smtp:
            smtp.starttls()
            smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
            print("[✅] Logged in to SMTP")

            for _, row in df.iterrows():
                name = row.get("Name", "there")
                to_email = str(row.get("Email", "")).strip()

                if not to_email or "@" not in to_email:
                    print(f"[⚠️] Skipping invalid or missing email for {name}: '{to_email}'")
                    fail_count += 1
                    continue

                msg = EmailMessage()
                msg["Subject"] = "Important: Food Support Inquiry"
                msg["From"] = EMAIL_ADDRESS
                msg["To"] = to_email
                msg.set_content(f"""Hi {name},

We're reaching out to support your food needs. Please reply "yes" to this message if any of the following statements apply to you:

• In the last 12 months, have you worried whether your food would run out before you had money to buy more?
• Did the food you bought just not last, and you didn't have money to get more?

Your answers will help us connect you with local food support resources.

Best regards,
Baylor Scott Nutrition Team
""")

                try:
                    smtp.send_message(msg)
                    print(f"[📤] Sent to {name} <{to_email}>")
                    sent_count += 1
                    time.sleep(1)  # prevent spam-like sending
                except Exception as e:
                    print(f"[❌] Failed to send to {to_email}: {e}")
                    fail_count += 1

        print(f"\n[📊] Summary: {sent_count} sent, {fail_count} failed.")
        print("[✅] Email sending completed.")
    except Exception as e:
        print(f"[❌] SMTP connection error: {e}")


In [ ]:
def send_followup(to_email, name="there"):
    try:
        with smtplib.SMTP(SMTP_SERVER, SMTP_PORT, timeout=10) as smtp:
            smtp.starttls()
            smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)

            msg = EmailMessage()
            msg["Subject"] = "Next Steps: Food Resource Support"
            msg["From"] = EMAIL_ADDRESS
            msg["To"] = to_email

            # Add Microsoft Forms link here
            form_link = ""  

            msg.set_content(f"""Hi {name},

Thank you for confirming your food support needs.

To connect you with the right local resources, please fill out this quick form:

👉 {form_link}

It will take less than 2 minutes to complete.

Best regards,  
Baylor Scott Nutrition Team
""")

            smtp.send_message(msg)
            print(f"[📨] Follow-up form sent to {to_email}")
    except Exception as e:
        print(f"[❌] Failed to send follow-up form to {to_email}: {e}")


In [6]:
def check_responses():
    df = load_patients()
    if df.empty:
        print("[ERROR] No patient data loaded.")
        return

    # Normalize Email and Food Insecurity column
    df["Email"] = df["Email"].astype(str).str.strip().str.lower()
    df["Food Insecurity"] = df.get("Food Insecurity", "").astype(str)

    try:
        print("[INFO] Connecting to IMAP...")
        mail = imaplib.IMAP4_SSL(IMAP_SERVER)
        mail.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        mail.select("inbox")

        print("[INFO] Searching for unseen replies...")
        status, messages = mail.search(None, '(UNSEEN SUBJECT "Important: Food Support Inquiry")')
        if status != "OK" or not messages[0]:
            print("[INFO] No new replies found.")
            mail.logout()
            return

        print(f"[INFO] Found {len(messages[0].split())} new replies.")
        for num in messages[0].split():
            status, data = mail.fetch(num, "(RFC822)")
            if status != "OK":
                print(f"[ERROR] Failed to fetch message {num}")
                continue

            raw_email = data[0][1]
            msg = email.message_from_bytes(raw_email)
            sender = email.utils.parseaddr(msg["From"])[1].strip().lower()
            body = ""

            if msg.is_multipart():
                for part in msg.walk():
                    if part.get_content_type() == "text/plain" and "attachment" not in str(part.get("Content-Disposition")):
                        body = part.get_payload(decode=True).decode(errors="ignore")
                        break
            else:
                body = msg.get_payload(decode=True).decode(errors="ignore")

            # Extract only the top reply line
            clean_response = body.split("\n")[0].strip().lower()
            print(f"[REPLY] {sender} ➜ \"{clean_response}\"")

            # Determine and log result
            positive_keywords = ["yes", "ok", "yeah"]
            match = df[df["Email"] == sender]

            if match.empty:
                print(f"[⚠️] Email not found in patient file: {sender}")
                continue

            if any(keyword in clean_response for keyword in positive_keywords):
                df.loc[df["Email"] == sender, "Food Insecurity"] = "Yes"
                name = match["Name"].values[0] if "Name" in match.columns else "there"
                print(f"[✅] Marked YES for {sender}")
                send_followup(sender, name=name)
            else:
                df.loc[df["Email"] == sender, "Food Insecurity"] = "No"
                print(f"[INFO] Marked NO for {sender}")

        # ✅ Save updates directly to sample.xlsx
        df.to_excel("sample.xlsx", index=False)
        mail.logout()
        print("[✅] Response check complete and saved to sample.xlsx.")

    except Exception as e:
        print(f"[❌ IMAP ERROR] {e}")


In [7]:
def merge_form_responses(form_file="Food Support Details(Sheet1) (2).csv"):
    try:
        print("[INFO] Loading original patient file...")
        patients_df = pd.read_excel("sample.xlsx")
        patients_df["Email"] = patients_df["Email"].astype(str).str.lower().str.strip()

        print(f"[INFO] Loading form responses from {form_file}...")
        if form_file.endswith(".csv"):
            form_df = pd.read_csv(form_file)
        else:
            form_df = pd.read_excel(form_file)

        # Clean column names
        form_df.rename(columns=lambda x: x.strip(), inplace=True)

        # Normalize form email column (handles variations like "Email", "Email1", etc.)
        email_column = next((col for col in form_df.columns if "email1" in col.lower()), None)
        if not email_column:
            print("[❌] No email column found in form response.")
            return

        form_df[email_column] = form_df[email_column].astype(str).str.lower().str.strip()

        # Define column mappings from form to patient Excel
        columns_to_copy = {
                "ZIP Code": "ZIP",
                "Preferred Contact Method (Text / Call)": "Preferred Contact Method",
                "Household Size": "Household Size",
                "Dietary Needs or Restrictions": "Dietary Needs",
                "Do you have access to transportation?": "Access to Transportation"
            }


        # Add missing columns to form responses if needed
        for form_col in columns_to_copy.keys():
            if form_col not in form_df.columns:
                print(f"[⚠️] Missing column in form: '{form_col}', adding empty column.")
                form_df[form_col] = ""

        # Add missing columns to patient Excel if needed
        for final_col in columns_to_copy.values():
            if final_col not in patients_df.columns:
                print(f"[ℹ️] Adding missing column in patient file: '{final_col}'")
                patients_df[final_col] = ""

        print("[INFO] Merging data...")
        for _, row in form_df.iterrows():
            email = row[email_column]
            if pd.isna(email) or "@" not in email:
                print(f"[⚠️] Skipping invalid email in form: {email}")
                continue

            match_idx = patients_df[patients_df["Email"] == email].index
            if not match_idx.empty:
                for form_col, final_col in columns_to_copy.items():
                    value = str(row.get(form_col, "")).strip()
                    patients_df[final_col] = patients_df[final_col].astype(str)
                    patients_df.loc[match_idx, final_col] = value
                print(f"[✅] Updated form data for: {email}")
            else:
                print(f"[⚠️] Email not found in main sheet: {email}")

        # ✅ Overwrite the original file
        patients_df.to_excel("sample.xlsx", index=False)
        print(f"[✅] Data merged and saved to sample.xlsx")

    except Exception as e:
        print(f"[❌] Merge failed: {e}")


In [8]:
send_emails()

[INFO] Excel file loaded successfully.
[INFO] Connecting to SMTP server...
[✅] Logged in to SMTP
[📤] Sent to Ali Khan <teilioteaser@gmail.com>

[📊] Summary: 1 sent, 0 failed.
[✅] Email sending completed.


In [10]:
check_responses()

[INFO] Excel file loaded successfully.
[INFO] Connecting to IMAP...
[INFO] Searching for unseen replies...
[INFO] Found 1 new replies.
[REPLY] teilioteaser@gmail.com ➜ "yes and yes."
[✅] Marked YES for teilioteaser@gmail.com
[📨] Follow-up form sent to teilioteaser@gmail.com
[✅] Response check complete and saved to sample.xlsx.


In [11]:
merge_form_responses()

[INFO] Loading original patient file...
[INFO] Loading form responses from Food Support Details(Sheet1) (2).csv...
[INFO] Merging data...
[✅] Updated form data for: teilioteaser@gmail.com
[✅] Data merged and saved to sample.xlsx


In [38]:
# 🔁 Optional Scheduler (Auto-check every 10 minutes)
def auto_check_scheduler(interval_minutes=10):
    print(f"[SCHEDULER] Starting auto-check every {interval_minutes} minutes...")
    while True:
        try:
            check_responses()
        except Exception as e:
            print(f"[SCHEDULER ERROR] {e}")
        time.sleep(interval_minutes * 60)